# Volume Forecast

- **V1**: 
  - By month and by year 
  - **Funded only**: given the used **MBA Forecast** is for **funded loans**
  - **Clean Clients only**: given it showed **highest correlation** between dollar amount and loan volume unit change (thus most confident results..)
- **V2**: 
    - LoanTypes doesn't make sense/doesn't affect seasonality (use loan purpose instead) - Margie
    - **Groupby : interest rates ranges + loan purpose** -> seasonality would be detected better this way - Margie
    - Need to break it into two (separately) : 
      - Loan Type Volumes is more for Employee (workload purposes..)
      1) Seasonality  
      2) Loan Type


## Imports

In [0]:
import numpy as np
import pandas as pd
import pyspark.pandas as ps
from pandas.tseries.offsets import BMonthEnd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import CustomBusinessDay, MonthEnd
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

import os
import sys

## Notebook imports
# nb path
sys.path.append(os.path.dirname(os.path.abspath('/Workspace/Shared/data_science/nexus_vision/Efficiency_Model_TDS1042')))
from input_data import read_sql_data, create_mba_forecast_df
import configs as c
from preprocess_data import convert_col_types, drop_nulls, create_ymd_cols, subset_data
from create_metrics import create_interest_rate_range_col, groupby_for_loans_amount_and_volume, calculate_percent_change, calulate_and_analyse_loan_volume_amount_correlations, calculate_cumulative_quarters

## Read Data

In [0]:
##TODO: delete below once imports... 

def read_sql_data(sandbox, folder, dataset):
    dataset_spark = spark.sql(f"SELECT * FROM {sandbox}.{folder}.{dataset}")
    dataset_pd = dataset_spark.toPandas()
    return dataset_pd

sandbox = "datause1_sandbox"
folder = "nexus_vision"
dataset1 = "ds_unique_loan_record"
dataset2 = "ds_data_and_ds_data_prior"

In [0]:
# Get UniqueLoanRecord - This dataset will be used to calculate loan fallouts 
# created by Margie - pulled by Sue
unique_loans_df = read_sql_data(sandbox, folder, dataset1)

# Get DSData = DSDataPrior + DSData - has all historical-Oct8th 
# created by Margie - pulled by Cameron
dsdata = read_sql_data(sandbox, folder, dataset2)

In [0]:
def create_mba_forecast_df(dollar_amount_list, quarters):
    # Create DF
    MBA_data = pd.DataFrame({
        'BillionDollars': dollar_amount_list,
        'Quarter': quarters
    })

    # Extract years and quarters from the input quarters list
    years = ['20' + q.split('_')[1] for q in quarters]  
    quarters = [q.split('_')[0] for q in quarters]  

    # Mapping the quarter strings to specific months
    quarter_to_month = {'Q1': '03', 'Q2': '06', 'Q3': '09', 'Q4': '12'}
    months = [quarter_to_month[q] for q in quarters]  # Last month of each quarter

    # Creating a new column with formatted dates
    MBA_data['FormattedQuarter'] = pd.to_datetime([f"{y}-{m}" for y, m in zip(years, months)])
    MBA_data.set_index('FormattedQuarter', inplace=True)

    # Normalize the billion-dollar amount to percentages
    MBA_data['NormalizedForecast'] = MBA_data['BillionDollars'].pct_change() * 100
    MBA_data = MBA_data.reset_index()

    # Let's only grab the columns we care about from MBA data
    MBA_forecast = MBA_data[['Quarter','FormattedQuarter', 'NormalizedForecast']]
    MBA_forecast = MBA_forecast.rename(columns={'FormattedQuarter':'YMD'})

    return MBA_forecast

In [0]:
## Define parameters, these will be periodically updated as needed by DS
dollar_amount_list = [333, 463, 444, 399, 422, 517, 543, 519]
quarters = ['Q1_23', 'Q2_23','Q3_23','Q4_23','Q1_24', 'Q2_24','Q3_24','Q4_24']
MBA_forecast = create_mba_forecast_df(dollar_amount_list, quarters)
MBA_forecast

,Quarter,YMD,NormalizedForecast
0,Q1_23,2023-03-01,NaN
1,Q2_23,2023-06-01,39.039039
2,Q3_23,2023-09-01,-4.103672
3,Q4_23,2023-12-01,-10.135135
4,Q1_24,2024-03-01,5.764411
5,Q2_24,2024-06-01,22.511848
6,Q3_24,2024-09-01,5.029014
7,Q4_24,2024-12-01,-4.419890


## Data Preprocessing

In [0]:
# #TODO: Delete below once imports...all below in preprocess_data nb
# # Define cols 
int_columns = [
        'IDKey', 'Clientkey', 'Applicationkey', 'BusinessDaysAppToFund', 'BusinessDaysApptoUWSub',
        'BusinessDaysApptoFinalApproval', 'BusinessDaysApptoClearToClose', 'BusinessDaysAppToProc',
        'CreditScore','CurrentMilestonekey', 'SelfEmployed','UWCondCR','UWTouches','LoanOfficerEmployeeKey',
        'LoanOfficerAsstEmployeeKey','ProcessorEmployeeKey','UnderwriterEmployeeKey','CloserEmployeeKey'
        ]
float_columns = [
        'LoanAmount','LTV','CLTV','Apprval','IntRate','PNIPmt','PITIPmt','HTI','DTI','GrossIncome',
        'LiquidAssets','CashfromBorrower'
        ] 
datetime_columns = [
        'ApplicationDate', 'SubmittoProc', 'UWSubmission', 'FinalApproval', 'Funded', 'FormattedQuarter', 'YMD'
        ]
def convert_col_types(df):
    # Convert columns types as necessary...
    for col in df.columns:
        if col in int_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(000).astype(int)
        elif col in float_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(000).astype(float)
        elif col in datetime_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df 


def drop_nulls(df, col):
    df = df[df[col]!='NULL']
    return df 

def create_ymd_cols(df, date_col, year=False, month=False, day=False, ymd=False):
    if year:
        df['year'] = df[date_col].dt.year
    if month:
        df['month'] = df[date_col].dt.month
    if day:
        df['day'] = df[date_col].dt.month
    if ymd:
        df['YMD'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
        df['YMD'] = pd.to_datetime(df['YMD'].dt.strftime('%Y-%m-%d'))
        # pd.to_datetime([f"{y}-{m}" for y, m in zip(years, months)])
    return df

## Only grab list of "good clients" -> provided Margie
clientkeys = [168, 192, 213, 218, 223, 245, 252, 255, 257, 258]
def subset_data(df, only_funded_loans=False, only_clean_clients=False, most_recent_year_data = False):
    if only_funded_loans:
        ## Subsetting: to ONLY funded loans
        df = df[df['Funded'].notnull()]

    if only_clean_clients:
        ## Subesetting: to df_funded to ONLY clean clients
        df = df[df['Clientkey'].isin(clientkeys)]
    
    if most_recent_year_data:
        df = df[df['year'] == df['year'].unique().max()]

    return df

In [0]:
## Drop NULL ApplicationDate -> Lead loans, do not want that in my data - Margie
dsdata = drop_nulls(dsdata, 'ApplicationDate')

## convert numeric cols dsdata
dsdata = convert_col_types(dsdata)

## Create year and month cols from ApplicationDate column
dsdata = create_ymd_cols(dsdata, 'ApplicationDate', year=True, month=True, day=False, ymd=False)

## Subset dataset as desired
funded_clean_clients_loans_dsdata = subset_data(dsdata, only_funded_loans=True, only_clean_clients=True)

In [0]:
#TODO: Delete below once imports...all below in create_metrics nb

# ranges, can change in config..
ranges = ['<3', '>=3', '>=4', '>=5', '>=6', '>=7']
def create_interest_rate_range_col(df, col):

    # Create IntRate conditions 
    conditions = [
    df[col] < 3,
    (df[col] >= 3) & (df[col] < 4),
    (df[col] >= 4) & (df[col] < 5),
    (df[col] >= 5) & (df[col] < 6),
    (df[col] >= 6) & (df[col] < 7),
    df[col] >= 7
    ]
    # Assign the ranges to 'IntRateRanges' based on conditions using numpy.select()
    df['IntRateRanges'] = np.select(conditions, ranges, default='Unknown')
    return df 

volume_groupby_cols = ['Clientkey','year', 'month']
loan_amount_groupby_cols = ['Clientkey','year', 'month','Applicationkey']
def groupby_for_loans_amount_and_volume(df):
    
    ## groupby to extract loan volume per client by month-year
    total_loan_volume_by_month_year = df.groupby(volume_groupby_cols)['Applicationkey'].nunique().reset_index(name='LoanVolume_my')

    ## groupby to extract loan amount per client by month-year
    unique_loan_amounts = df.groupby(loan_amount_groupby_cols)['LoanAmount'].last().reset_index()
    total_loan_amount_by_month_year = unique_loan_amounts.groupby(volume_groupby_cols)['LoanAmount'].sum().reset_index(name='TotalLoanAmount_my')

    ## create dataframe that outputs each client, along with their loan metrics..
    loan_totals_by_month_year_ = [total_loan_volume_by_month_year, total_loan_amount_by_month_year]
    loan_totals_by_month_year = reduce(lambda  left, right: pd.merge(left, right, on=volume_groupby_cols, how='left'), loan_totals_by_month_year_)
    return loan_totals_by_month_year

def calculate_percent_change(group):
    ## clacluate percent change for LoanAmount and LoanVolume for correlation calculation..
    group['percent_change_LoanVolume'] = group['LoanVolume_my'].pct_change()*100
    group['percent_change_TotalLoanAmount'] = group['TotalLoanAmount_my'].pct_change()*100
    return group

def calulate_and_analyse_loan_volume_amount_correlations(df):
    ## Function purpose: calculates correlation between loan dollar amount and loan volume w.r.t. both's unit change (percent increase/decrease)
    ## Function output: a message indicating if we have a high positive, negative, or neutral (below 0.7) correlation between the two variables

    ## Groupby and get Loan Amount and Volume totals
    loan_totals_by_month_year = groupby_for_loans_amount_and_volume(df)

    # Calculate percent-change of Loan Amount and Loan Volume and compare correlation..
    loan_totals_by_month_year = loan_totals_by_month_year.groupby('Clientkey').apply(calculate_percent_change)
    loan_totals_by_month_year_last_year_data = subset_data(loan_totals_by_month_year, only_funded_loans=False, only_clean_clients=False, most_recent_year_data=True)

    ## create correlation dataframes 
    loan_totals_by_month_year_corr = loan_totals_by_month_year.corr()
    # sub to 2023 data since that's what we'll use for forecasts...
    loan_totals_by_month_year_last_year_data_corr = loan_totals_by_month_year_last_year_data.corr()


    # Extract correlation value between the specified columns, for all data + most recent also
    correlation_value_all_years = loan_totals_by_month_year_corr.loc['percent_change_LoanVolume', 'percent_change_TotalLoanAmount']
    correlation_value_last_year = loan_totals_by_month_year_last_year_data_corr.loc['percent_change_LoanVolume', 'percent_change_TotalLoanAmount']


    # Determine the nature of the correlation for both correlations (all years + last year)
    ## all years
    if correlation_value_all_years > 0.7:
        correlation_type = "high positive"
    elif correlation_value_all_years < 0:
        correlation_type = "negative"
    else:
        correlation_type = "neutral"

    # last year
    if correlation_value_last_year > 0.7:
        correlation_type_last_year = "high positive"
    elif correlation_value_last_year < 0:
        correlation_type_last_year = "negative"
    else:
        correlation_type_last_year = "neutral"

    # Provide analysis message
    analysis_message = f"Correlations between the percent change of the total loans' dollar amount and percent change of loan volume are {correlation_value_all_years:.3f}, and {correlation_type}.\nLast year's correlations between the two variables are {correlation_value_last_year:.3f}, and {correlation_type_last_year}."

    return print(analysis_message)

In [0]:
## Create interest rate ranges, to include with volume forecast 
funded_clean_clients_loans_dsdata = create_interest_rate_range_col(funded_clean_clients_loans_dsdata, 'IntRate')

## Let's look at correlations between loan dollar amount and loan volume w.r.t. both's unit change (percent increase/decrease) 
calulate_and_analyse_loan_volume_amount_correlations(funded_clean_clients_loans_dsdata)

Correlations between the percent change of the total loans' dollar amount and percent change of loan volume are 0.738, and high positive.
Last year's correlations between the two variables are 0.951, and high positive.


So moving forward, based on the above correlations..:
1) When using MBA Forecast data, we will assume, **unit increase in $$ amount -equiv- to unit increase in Application Volume**
2) We will (evidently) be **more confident in our Clean-Client forecasts**...so that will be V1 forecast (for the sake of *validation*)

## Skeleton Volume Forecast
- Predict Volume Based on **histoical data + Seasonality Adjustment**
- V1: **Forecasting Funded Loans Volumes** 

### Forecasting for Loan Application Volume

- Assumptions:
  - **MBA Mort.Loan Originations Forecasts** are relatively close to expected
  - Unit change in dollar equiv. to unit change in Volume  (direct high positive correlation)

- Logic: 
  - Translated MBA quarterly forecasts to percentage change
  - Used **last quarter of Q3_2023 real data** (cumsum) and applied forecasts
  - To get monthly forecasts, the quarterly's were split (per David H's rec)
  
  - consequently monthly's for the same quarter will appear to be a straight line

In [0]:
## TODO: get rid of below functions once imports complete from create_metrics.py, preprocess_data.py...
## Get the cum-sum at the end of each quarter, to apply quarterly forecasts later...
def calculate_cumulative_quarters(group):
    # Sort the group by 'YMD' before calculating cumulative sum
    group = group.sort_values('YMD')
    
    # Calculate cumulative sum for LoanVolume_my and TotalLoanAmount_my, resetting at the start of each quarter
    group['LoanVolume_my_Q'] = group.groupby(group['YMD'].dt.to_period("Q")).cumsum()['LoanVolume_my']
    group['TotalLoanAmount_my_Q'] = group.groupby(group['YMD'].dt.to_period("Q")).cumsum()['TotalLoanAmount_my']
    
    return group

In [0]:
def prepare_data_for_forecast(funded_clean_clients_loans_dsdata):
    ## Format date-YMD column
    funded_clean_clients_loans_totals = groupby_for_loans_amount_and_volume(funded_clean_clients_loans_dsdata)
    funded_clean_clients_loans_totals = create_ymd_cols(funded_clean_clients_loans_totals, _, year=False, month=False, day=False, ymd=True)

    ## Subset the data we will apply forecast to - ONLY most recent year data
    funded_clean_clients_loans_totals_recent_year = subset_data(funded_clean_clients_loans_totals, only_funded_loans=False, only_clean_clients=False, most_recent_year_data=True)
    # Subset to only colmns we care about #TODO: verify desired output is NOT full df
    funded_clean_clients_loans_totals_recent_year = funded_clean_clients_loans_totals_recent_year[['YMD', 'Clientkey','TotalLoanAmount_my', 'LoanVolume_my']]

    df_to_forecast = funded_clean_clients_loans_totals_recent_year.groupby('Clientkey').apply(calculate_cumulative_quarters).reset_index(drop=True)

    return df_to_forecast

In [0]:
data_to_forecast = prepare_data_for_forecast(funded_clean_clients_loans_dsdata)

In [0]:
display(data_to_forecast)

YMD,Clientkey,TotalLoanAmount_my,LoanVolume_my,LoanVolume_my_Q,TotalLoanAmount_my_Q
2023-01-01T00:00:00Z,168,1.67030221E8,582,582,1.67030221E8
2023-02-01T00:00:00Z,168,1.94130605E8,683,1265,3.61160826E8
2023-03-01T00:00:00Z,168,2.34539107E8,810,2075,5.95699933E8
2023-04-01T00:00:00Z,168,2.0324397E8,703,703,2.0324397E8
2023-05-01T00:00:00Z,168,2.17563425E8,770,1473,4.20807395E8
2023-06-01T00:00:00Z,168,2.11710413E8,740,2213,6.32517808E8
2023-07-01T00:00:00Z,168,1.90024496E8,672,672,1.90024496E8
2023-08-01T00:00:00Z,168,1.7758252E8,647,1319,3.67607016E8
2023-09-01T00:00:00Z,168,1.42814723E8,532,1851,5.10421739E8
2023-10-01T00:00:00Z,168,1.12241659E8,410,410,1.12241659E8


In [0]:
def apply_mba_forecast(df_to_forecast, MBA_data, last_desired_forecast_date):
    
    ## Filter MBA forecast data for the relevant time period
    MBA_forecast = MBA_data[MBA_data['YMD'] >= pd.to_datetime(last_desired_forecast_date)] #'2023-09-01'

    # Initialize empty DataFrames to store forecast results
    forecasted_dataframes = []

    # Group by Clientkey and apply forecast for each group
    for clientkey, group in df_to_forecast.groupby('Clientkey'):
        # Merge the MBA forecast data with the loan totals data on 'YMD' column
        merged_data = pd.merge(group, MBA_forecast, on='YMD', how='right')
        merged_data['Clientkey'] = merged_data['Clientkey'].iloc[0]
        
        # Initialize LoanVolume Forecast column
        merged_data['LoanVolume_Forecast'] = 0
        # Set initial value based on Q data
        merged_data['LoanVolume_Forecast'].iloc[0] = merged_data['LoanVolume_my_Q'].iloc[0]

        # Apply LoanVolume Forecast from MBA
        for i in range(1, len(merged_data)):
            merged_data['LoanVolume_Forecast'].iloc[i] = merged_data['LoanVolume_Forecast'].iloc[i-1] * (1 + merged_data['NormalizedForecast'].iloc[i]/100)

        # Initialize LoanAmount Forecast column
        merged_data['LoanAmount_Forecast'] = 0
        # Set initial value based on Q data
        merged_data['LoanAmount_Forecast'].iloc[0] = merged_data['TotalLoanAmount_my_Q'].iloc[0]

        # Apply LoanAmount Forecast from MBA
        for i in range(1, len(merged_data)):
            if not pd.isnull(merged_data['LoanAmount_Forecast'].iloc[i-1]):
                merged_data['LoanAmount_Forecast'].iloc[i] = merged_data['LoanAmount_Forecast'].iloc[i-1] * (1 + merged_data['NormalizedForecast'].iloc[i]/100)

        # # Translate into monthly data 
        merged_data['LoanAmountForecast_m'] = np.round(merged_data['LoanAmount_Forecast'] / 3, 4)
        merged_data['LoanVolumeForecast_m'] = np.round(merged_data['LoanVolume_Forecast'] / 3, 4)
        
        # Append the forecasted dataframe for the current Clientkey to the list
        forecasted_dataframes.append(merged_data)
    
    # Concatenate all forecasted dataframes
    forecasted_data = pd.concat(forecasted_dataframes)
    
    return forecasted_data


In [0]:
all_forecasted = apply_mba_forecast(data_to_forecast, MBA_forecast, '2023-09-01')
all_forecasted[all_forecasted['Clientkey']==168]

,YMD,Clientkey,TotalLoanAmount_my,LoanVolume_my,LoanVolume_my_Q,TotalLoanAmount_my_Q,Quarter,NormalizedForecast,LoanVolume_Forecast,LoanAmount_Forecast,LoanAmountForecast_m,LoanVolumeForecast_m
0,2023-09-01,168.0,142814723.0,532.0,1851.0,510421739.0,Q3_23,-4.103672,1851.000000,5.104217e+08,1.701406e+08,617.0000
1,2023-12-01,168.0,NaN,NaN,NaN,NaN,Q4_23,-10.135135,1663.398649,4.586898e+08,1.528966e+08,554.4662
2,2024-03-01,168.0,NaN,NaN,NaN,NaN,Q1_24,5.764411,1759.283784,4.851306e+08,1.617102e+08,586.4279
3,2024-06-01,168.0,NaN,NaN,NaN,NaN,Q2_24,22.511848,2155.331081,5.943424e+08,1.981141e+08,718.4437
4,2024-09-01,168.0,NaN,NaN,NaN,NaN,Q3_24,5.029014,2263.722973,6.242320e+08,2.080773e+08,754.5743
5,2024-12-01,168.0,NaN,NaN,NaN,NaN,Q4_24,-4.419890,2163.668919,5.966416e+08,1.988805e+08,721.2230


## Loan Fallout 
- **V1**: 
    - Calculate fallout % for each of the desired Milestones
    - Done for clean clients 
- **V2:** 
    - By **Loan Purpose/Loan Type/Interest Rates...**?
    - **Separate Purchase and Refinance** for fallout
- **Out of Scope**: How can we best leverage **Lock Date** and time spent locked/unlcoked for most meaningful/interpretable outputs...

- Milestones we care about:
    - Application (none)
    - Submit to Processing (processor)
    - Submit to Underwriting (UW)
    - Final Approved/Clear to Close - interchangable (closer)

- How can we safely assume, that once a loan makes it to a stage and no further, that it has indeed fallen-out?
    - Logic will be provided to determine *inactive loans* (by Margie/David)...


**Analysis**:... 
- use **DSUniqueLoanRecord** - for clean milestone data 
  - **DO NOT use CurrentMilestone** column, but rather the columns with desired milestones instead...
- **subset to clean clients** (ensure all have desired Milestones..) 
  - analysis on clients that do not have desired milstones...if any..? 
- **go further than 90days ago for fallout** (otherwise doesn't make sense)
  - stop at september 23 (don't have enough to know)
  - subset to 6-months: applicationdata > 01/01/23 < 07/01/23
  - to start, I will **calculate fallouts for data > 2022**
- **remove Purpose = 'C'** (Construction Loans take hella long...can't accuratly calculate fallout)

- if Final Approval DNE = Clear to Close = Submitted to Closing (COALESC Final Approval + Clear to Close)
- **verify**: *unique loan counts* is what we're intrested in -> Margie - yes:)
  - **i.o.w. if we care about calculating how much work each role is getting, we should care about duplicates in LoanNumber?** -- ask David/Margie -> No, unique LoanNumber is what we want  
- **globally or per FI** ...
  - start latter then work backwards -> iff different across FIs then FI level..

In [0]:
## clean data 
unique_loans_df_clean = subset_data(unique_loans_df, only_funded_loans=False, only_clean_clients=True, most_recent_year_data=False):

# ## Before any further analysis, need to subset data to all before '07/01/23' (since need at least 6-months of data..)
clean_uni_fallout = clean_uni[(clean_uni['ApplicationDate'] > pd.to_datetime('01/01/22'))&(clean_uni['ApplicationDate'] < pd.to_datetime('07/01/23'))]
# Verify if we ONLY want to use 22+23 data 
# 

## Now remove Purpose == 'Construction' (since construction Loans length > 6months )
clean_uni_fallout = clean_uni_fallout[clean_uni_fallout['Purpose']!='Construction']

/root/.ipykernel/995/command-4281590173744933-3448837926:5: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  clean_uni_fallout = clean_uni[(clean_uni['ApplicationDate'] > pd.to_datetime('01/01/22'))&(clean_uni['ApplicationDate'] < pd.to_datetime('07/01/23'))]


In [0]:
## Applying: COALESCE(FinalApproval, ClearToClose)..
clean_uni_fallout['FinalApproval'].fillna(clean_uni_fallout['ClearToClose'], inplace=True)

# given above, I will no longer need the ClearToClose Milestone...

**Verify** 
- okey to combine CleartoClose into FinalApproval..to unify analysis:)
- year to subset the data...

In [0]:
clean_uni_fallout.groupby(['Clientkey'])[['ApplicationDate', 'SubmittoProc', 'UWSubmission', 'FinalApproval', 'ClearToClose', 'Funded']].apply(lambda x: x.notnull().sum()).reset_index()

,Clientkey,ApplicationDate,SubmittoProc,UWSubmission,FinalApproval,ClearToClose,Funded
0,168,21320,17964,16795,14338,14108,14156
1,192,4790,3838,1163,2999,2999,3008
2,213,4614,4376,3974,3679,0,3627
3,218,17184,13407,12554,11718,11718,11590
4,223,16511,13133,12857,12754,11498,11398
5,245,3188,2907,2910,2806,2792,2768
6,252,1686,1377,1297,1146,1146,1146
7,255,12123,9597,9058,6755,6755,6669
8,257,4354,3565,3363,2938,2938,3064
9,258,1924,1656,1519,172,172,1336


In [0]:
## Lets look at the distribution of Milestones across Clients (clean clients)
milestones_across_clean_FIs = clean_uni_fallout.groupby(['Clientkey'])[['ApplicationDate', 'SubmittoProc', 'UWSubmission', 'FinalApproval', 'Funded']].apply(lambda x: x.notnull().sum()).reset_index()

# no need for nunique Loans here: 
# since only unique records are in the used DF

display(milestones_across_clean_FIs) 

Clientkey,ApplicationDate,SubmittoProc,UWSubmission,FinalApproval,Funded
168,21320,17964,16795,14338,14156
192,4790,3838,1163,2999,3008
213,4614,4376,3974,3679,3627
218,17184,13407,12554,11718,11590
223,16511,13133,12857,12754,11398
245,3188,2907,2910,2806,2768
252,1686,1377,1297,1146,1146
255,12123,9597,9058,6755,6669
257,4354,3565,3363,2938,3064
258,1924,1656,1519,172,1336


Databricks visualization. Run in Databricks to view.

Observe above distributions...
- For the most part distribution look the same
- Though **not all clients use UWSubmission & SubmittoProc similarly**, those two Milestones' distributions has more variability between FIs

- From above, **fallout rates will be calculated independently for FIs** rather than globally across FIs 

### Loan Fallout Calculation
Process: 
- Calculate fallout for a clients, for each of the desired milestones
- Apply fallouts to Monthly forcasted funded-loans (to get the full numbers of Application forecasts)
    - **how are we going to apply fallouts for forecasted?**
    - **should we be ab le to *forecast* fallouts? or expect them to follow same behavior?** - Analysis...

- Methods:
    - calculate fallout percentages for each of the loan stages
    - fallouts are percentages, calculated monthly, independently for the different years (data used spans 01-2022 to 06-2023)
    - the output, is a dataframe with the percentage fallouts, for each of the milestones, for the different clients


In [0]:
def calculate_monthly_fallout_by_client(df):
    # Convert dates to datetime
    df['ApplicationDate'] = pd.to_datetime(df['ApplicationDate'])
    df['SubmittoProc'] = pd.to_datetime(df['SubmittoProc']) 
    df['UWSubmission'] = pd.to_datetime(df['UWSubmission'])
    df['FinalApproval'] = pd.to_datetime(df['FinalApproval'])
    df['Funded'] = pd.to_datetime(df['Funded'])
    
    # Group by Clientkey
    grouped = df.groupby('Clientkey') 
    
    # Initialize output dict
    out = {}
    
    # Iterate through each group 
    for name, group in grouped:
        # Get distinct months 
        months = group['ApplicationDate'].dt.to_period('M').unique()
        
        # Initialize dataframe
        out[name] = pd.DataFrame(index=months)

        # Iterate through stages
        for col in ['SubmittoProc', 'UWSubmission', 'FinalApproval', 'Funded']:

            # Groupby month and count fallouts
            grp = group.groupby(group['ApplicationDate'].dt.to_period('M'))[col].apply(lambda x: x.isnull().sum())

            # Calculate percentages
            fallout_pct = grp / group.groupby(group['ApplicationDate'].dt.to_period('M'))['LoanNumber'].count()

            # Add to output dataframe
            out[name][col] = fallout_pct
            
    ## append items to df
    client_dfs = {}
    for name, df in out.items():
        df = df.reset_index().melt(id_vars='ApplicationDate')
        df['Client'] = name
        client_dfs[name] = df

    output_df = pd.concat(client_dfs.values(), ignore_index=True)
    output_df.columns = ['Month', 'Stage', 'FalloutPct', 'Client']
    
    return output_df
    
    # return out

In [0]:
clean_clients_fallouts #make into one dataframe 

{168:          SubmittoProc  UWSubmission  FinalApproval    Funded
 2023-03      0.159967      0.215751       0.328958  0.339623
 2023-05      0.158992      0.209383       0.320591  0.334492
 2023-04      0.157184      0.201543       0.318226  0.327869
 2023-06      0.159817      0.193607       0.317808  0.325114
 2022-10      0.171533      0.265207       0.367397  0.369830
 2022-09      0.167677      0.225253       0.338384  0.351515
 2022-11      0.164609      0.222222       0.333333  0.344307
 2022-12      0.143317      0.190016       0.315620  0.326892
 2023-01      0.150000      0.197619       0.305952  0.311905
 2023-02      0.161227      0.204748       0.322453  0.327399
 2022-03      0.163021      0.220312       0.329688  0.338542
 2022-02      0.153565      0.207191       0.331505  0.334552
 2022-01      0.149020      0.197199       0.319888  0.331092
 2022-04      0.158606      0.215505       0.333570  0.338549
 2022-06      0.149635      0.221898       0.338686  0.348175
 20

In [0]:
## Run fallout calculation function 
clean_clients_fallouts = calculate_monthly_fallout_by_client(clean_uni_fallout)

## show example output 
clean_clients_fallouts[192]

,SubmittoProc,UWSubmission,FinalApproval,Funded
2023-04,0.137184,0.216606,0.321300,0.328520
2023-05,0.157377,0.213115,0.400000,0.403279
2023-03,0.151394,0.231076,0.342629,0.346614
2023-06,0.133603,0.174089,0.340081,0.356275
2022-11,0.267677,0.974747,0.459596,0.404040
2023-02,0.154639,0.237113,0.335052,0.324742
2022-09,0.218605,0.972093,0.381395,0.381395
2022-10,0.308642,0.944444,0.444444,0.444444
2022-12,0.178344,0.980892,0.305732,0.312102
2023-01,0.196429,0.486607,0.370536,0.321429


In [0]:
## this is just to show outputs, and won't be automated unless otherwise specified...
fallout168 = clean_clients_fallouts[192].reset_index()
fallout168['index'] = fallout168['index'].astype(str)

display(fallout168)

index,SubmittoProc,UWSubmission,FinalApproval,Funded
2023-04,0.1371841155234657,0.21660649819494585,0.3212996389891697,0.3285198555956679
2023-05,0.15737704918032788,0.21311475409836064,0.4,0.40327868852459015
2023-03,0.15139442231075698,0.23107569721115537,0.3426294820717131,0.3466135458167331
2023-06,0.13360323886639677,0.17408906882591094,0.340080971659919,0.3562753036437247
2022-11,0.2676767676767677,0.9747474747474747,0.4595959595959596,0.40404040404040403
2023-02,0.15463917525773196,0.23711340206185566,0.33505154639175255,0.3247422680412371
2022-09,0.2186046511627907,0.9720930232558139,0.3813953488372093,0.3813953488372093
2022-10,0.30864197530864196,0.9444444444444444,0.4444444444444444,0.4444444444444444
2022-12,0.17834394904458598,0.9808917197452229,0.3057324840764331,0.31210191082802546
2023-01,0.19642857142857142,0.48660714285714285,0.3705357142857143,0.32142857142857145


Databricks visualization. Run in Databricks to view.

#### Analysis on fallout 
 - we need to figure out **if fallouts are stable across months**
    - should we use **average monthly fallouts** for forcasts?
    - or **forecast monthly-fallouts trends?**

In [0]:
def analyze_monthly_fallout_stability(client_fallouts, clientkey):

    print(f"Analyzing monthly fallout stability for client: {clientkey}")
    df = client_fallouts[clientkey]

     # Sort index by date
    df = df.reset_index()  
    df = df.sort_values('index')
    df = df.set_index('index')

    # Calculate rolling stats
    roll_mean = df.rolling(window=3).mean()
    roll_std = df.rolling(window=3).std()
    
    # Check if stdev is within 10% of mean 
    stable_cols = roll_std.columns[roll_std.max() < 0.16*roll_mean.min()]

    # Create dataframes
    roll_mean_df = pd.DataFrame(roll_mean)
    roll_std_df = pd.DataFrame(roll_std)
    stable_cols_df = pd.DataFrame({'Stable Fallout Columns': stable_cols})

    # Check if stdev is within 16% of mean (arbitrary threshold)
    if len(stable_cols) > 0:
        print(f"\nFallouts in columns {list(stable_cols)} seem stable enough to use average fallouts.")
    else:
        print("\nNo fallout columns appear stable enough to use average fallouts.")

    return roll_mean_df, roll_std_df, stable_cols_df

In [0]:
mean_df, std_df, stable_df = analyze_monthly_fallout_stability(clean_clients_fallouts, 192)

Analyzing monthly fallout stability for client: 192

No fallout columns appear stable enough to use average fallouts.


In [0]:
mean_df
### indicate assumptions : past 6mo is indictive of next 12mo...

## i will wait to apply this to forecast till i clean/functionalize forecast code

,SubmittoProc,UWSubmission,FinalApproval,Funded
index,,,,
2022-01,NaN,NaN,NaN,NaN
2022-02,NaN,NaN,NaN,NaN
2022-03,0.188392,1.000000,0.346833,0.348611
2022-04,0.201928,1.000000,0.380221,0.381998
2022-05,0.200070,1.000000,0.380228,0.385430
2022-06,0.224008,1.000000,0.414439,0.417864
2022-07,0.221485,0.986885,0.394031,0.397456
2022-08,0.233241,0.974456,0.392708,0.393838
2022-09,0.230616,0.965154,0.381628,0.382758


**Rolling 3-month Mean DataFrame** - This shows the 3-month rolling average fallout rate for each stage. - Look at the trend in means over time - a flat trend indicates stable fallout rates. An increasing or decreasing trend indicates changing fallout rates. - A flat trend supports using the average fallout rate for forecasting. A changing trend suggests forecasting monthly trends may be better. 

**Rolling 3-month Std Dev DataFrame** - This shows the 3-month rolling standard deviation in fallout rates. - Lower standard deviation indicates more stable fallout rates. Higher standard deviation indicates more variability/volatility. - Low, stable STD across time supports using average fallouts for forecasting. High or increasing STD suggests monthly trends may need to be forecasted. 

**Stable Columns DataFrame** - This identifies columns where the STD is within 10% of the mean. - Columns listed here have relatively low variability compared to the mean. - These columns likely have stable enough fallout rates to use the averages. 

Columns not listed may require forecasting monthly trends. 

**Interpretation** - 

- Flat rolling means: low/stable rolling STD, and many columns listed as "stable" support the hypothesis that monthly fallouts are stable enough to use average rates for forecasting. -

- Increasing/decreasing rolling means, high/increasing rolling STD, and few stable columns suggests monthly fallout trends may need to be forecasted rather than just using the averages. 

So in summary, these dataframes allow assessing the stability of monthly fallouts to determine whether averages or monthly trends would be better for forecasting. Flat trends in the means, low STD, and many stable columns supports using averages.

In [0]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

def analyze_fallout_stability(df):
  
#   print('Original DataFrame:')
#   print(df.head())
  
  # Calculate rolling averages
#   df = df.set_index('SubmittoProc')
  for col in df.columns:
    df[f'{col}_roll'] = df[col].rolling(3).mean()
  
  print('\nDataFrame with Rolling Averages:')
  print(df.head())

  # Visualize 
  plt.figure(figsize=(10, 5))
  df.plot(y=['Funded', 'Funded_roll'])
  plt.title('Funded Fallout Rate for Client 168')
  plt.ylabel('Fallout %')
  plt.show()

  # Statistical tests
  for col in df.columns:
    result = adfuller(df[f'{col}_roll'])
    print(f'ADF Statistic for {col}: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical Values: {result[4]}')

  print('\nCoefficient of Variation:')
  print(df.std() / df.mean())

  # Compare across years
  df['Year'] = df.index.year
  yearly_avg = df.groupby('Year').mean()
  print('\nYearly Averages:')
  print(yearly_avg)

In [0]:
# Forecast subsequent rows based on previous forecast
for i in range(1, len(MBA_clean_clients_loan_totals)):
  
  if pd.isnull(MBA_clean_clients_loan_totals['LoanAmount_Forecast'].iloc[i]):
    forecast_pct = 1 + MBA_clean_clients_loan_totals['NormalizedForecast_m'].iloc[i]/100
    prev_forecast = MBA_clean_clients_loan_totals['LoanAmount_Forecast'].iloc[i-1]
    forecast_amount = prev_forecast * forecast_pct
    
    MBA_clean_clients_loan_totals['LoanAmount_Forecast'].iloc[i] = forecast_amount

  if pd.isnull(MBA_clean_clients_loan_totals['LoanVolume_Forecast'].iloc[i]):  
    forecast_pct = 1 + MBA_clean_clients_loan_totals['NormalizedForecast_m'].iloc[i]/100
    prev_forecast = MBA_clean_clients_loan_totals['LoanVolume_Forecast'].iloc[i-1]  
    forecast_volume = prev_forecast * forecast_pct

    MBA_clean_clients_loan_totals['LoanVolume_Forecast'].iloc[i] = forecast_volume

# Format date    
MBA_clean_clients_loan_totals['YMD'] = MBA_clean_clients_loan_totals['YMD'].dt.strftime('%Y-%m-%d') 

# Subset data
MBA_clean_clients_loan_totals_sub = MBA_clean_clients_loan_totals[~MBA_clean_clients_loan_totals['Quarter'].str.contains('25')]

display(MBA_clean_clients_loan_totals_sub)